### **Import all packages** ##

In [7]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# Set style for better looking plots
plt.style.use('default')
%matplotlib inline

print("All packages imported successfully!")

All packages imported successfully!


### **Load objectives data**

In [6]:
df = pd.read_csv(r'C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\10_Analysis\1002_PCA\1002_All_objectives.csv', 
                 sep=';', 
                 index_col='sim')

print("Objectives Data Shape:", df.shape)
print("\nFirst 5 rows of objectives data:")
display(df.head())

# Display basic statistics
print("\nBasic statistics of objectives data:")
display(df.describe())

# Identify pollutant columns
pollutant_columns = ['CO2', 'TSS', 'TP', 'TN', 'Cu', 'Pb', 'Zn']
print("Data for sim42 (the outlier):")
print(df.loc['sim42', pollutant_columns])

# Create a clean copy of the entire DataFrame
df_clean = df.copy()

# Apply capping to handle outliers in pollutant columns
for col in pollutant_columns:
    upper_cap = df_clean[col].quantile(0.99)
    lower_cap = df_clean[col].quantile(0.01)
    df_clean[col] = df_clean[col].clip(lower=lower_cap, upper=upper_cap)

print("\nAfter capping - value for sim42 TSS:", df_clean.loc['sim42', 'TSS'])
print("After capping - value for sim42 CO2:", df_clean.loc['sim42', 'CO2'])

Objectives Data Shape: (150, 19)

First 5 rows of objectives data:


PR2   PR5  PR10  PR20    TR   PHI    Temp   BGF          Inv  \
sim                                                                   
sim1  0.00  0.00  0.00  0.00  0.00  0.00  0.0000  0.00        0.000   
sim2  0.29  0.24  0.25  0.28  0.59  0.55  0.0020  0.45  2275334.205   
sim3  0.40  0.29  0.25  0.36  0.36  0.34  0.0012  0.22  1386333.500   
sim4  0.22  0.22  0.25  0.23  0.44  0.41  0.0016  0.24  2046027.988   
sim5  0.46  0.33  0.27  0.33  0.40  0.36  0.0023  0.42  2113380.266   

             Maint         UNA           GA          CO2   TSS    TP    TN  \
sim                                                                          
sim1       0.00000    0.000000     0.000000     0.000000  0.00  0.00  0.00   
sim2  122996.97850  105.342624  1614.908265  3148.518124  0.83  0.79  0.80   
sim3   76818.38861   82.742087  1069.562179   895.855142  0.71  0.71  0.71   
sim4  117080.85560   73.216244  1554.823203  1050.302508  0.45  0.40  0.41   
sim5  119223.78170   98.284197  1578.227187  3238.029588  0.96  0.92  0.94   

        Cu    Pb    Zn  
sim                     
sim1  0.00  0.00  0.00  
sim2  0.81  0.83  0.83  
sim3  0.71  0.71  0.71  
sim4  0.42  0.45  0.45  
sim5  0.94  0.96  0.96


Basic statistics of objectives data:


PR2         PR5        PR10       PR20          TR         PHI  \
count  150.000000  150.000000  150.000000  150.00000  150.000000  150.000000   
mean     0.338867    0.279000    0.283200    0.31240    0.418667    0.389467   
std      0.147787    0.139128    0.136619    0.13848    0.127493    0.115937   
min     -0.020000   -0.020000   -0.010000   -0.02000    0.000000    0.000000   
25%      0.240000    0.200000    0.200000    0.22000    0.330000    0.320000   
50%      0.330000    0.270000    0.280000    0.32000    0.430000    0.390000   
75%      0.440000    0.360000    0.370000    0.41750    0.507500    0.475000   
max      0.810000    0.720000    0.710000    0.69000    0.720000    0.640000   

             Temp         BGF           Inv          Maint         UNA  \
count  150.000000  150.000000  1.500000e+02     150.000000  150.000000   
mean     0.001907    0.337933  2.087523e+06  119473.435617   74.747369   
std      0.000638    0.147936  9.763774e+05   59170.851840   19.632477   
min      0.000000    0.000000  0.000000e+00       0.000000    0.000000   
25%      0.001600    0.230000  1.388448e+06   77056.584015   62.792727   
50%      0.002000    0.315000  2.072482e+06  118223.730650   73.264760   
75%      0.002300    0.437500  2.725578e+06  158099.009875   88.749441   
max      0.003500    0.720000  4.054589e+06  235240.625400  113.552311   

                GA          CO2       TSS          TP          TN          Cu  \
count   150.000000   150.000000  150.0000  150.000000  150.000000  150.000000   
mean   1575.257126  2209.143050    0.6204    0.559600    0.579800    0.585067   
std     744.209130  1476.004562    0.4665    0.488493    0.476312    0.474430   
min       0.000000     0.000000   -3.5600   -3.560000   -3.560000   -3.560000   
25%    1034.613167  1131.541054    0.4825    0.430000    0.432500    0.440000   
50%    1554.078230  1856.002862    0.7350    0.685000    0.695000    0.705000   
75%    2064.411671  3099.275358    0.8975    0.840000    0.857500    0.867500   
max    3013.974665  5989.047490    1.0000    1.000000    1.000000    1.000000   

               Pb          Zn  
count  150.000000  150.000000  
mean     0.620267    0.620933  
std      0.467257    0.466438  
min     -3.560000   -3.560000  
25%      0.480000    0.482500  
50%      0.740000    0.740000  
75%      0.890000    0.900000  
max      1.000000    1.000000

Data for sim42 (the outlier):
CO2    368.124472
TSS     -3.560000
TP      -3.560000
TN      -3.560000
Cu      -3.560000
Pb      -3.560000
Zn      -3.560000
Name: sim42, dtype: float64

After capping - value for sim42 TSS: -0.5224
After capping - value for sim42 CO2: 368.1244718


### **Load and Prepare Input Parameters**

In [8]:
# Load your LID scenarios file
input_params_df = pd.read_csv(r'C:\Users\ABI\OneDrive - NIVA\Documents\GitHub\SWMM_MOO\01_Preprocessing\0103_Data_cleaned_random_generated_scenarios.csv', 
                             sep=';', 
                             index_col='sim')

print("Input parameters shape:", input_params_df.shape)
print("\nFirst 5 rows of input parameters:")
display(input_params_df.head())

# Extract only the columns we need (LID implementation areas)
# Exclude the '*_Aimp' columns and focus on the area columns
area_columns = [col for col in input_params_df.columns if not col.endswith('_Aimp') and col not in ['Type']]
print(f"\nLID Area columns to use: {area_columns}")

# Keep the 'Type' column and area columns, then convert categorical 'Type' to numerical
X_inputs = input_params_df[['Type'] + area_columns].copy()
X_inputs = pd.get_dummies(X_inputs, columns=['Type'], prefix='Type')

print("\nProcessed input features for regression:")
print("Shape:", X_inputs.shape)
display(X_inputs.head())

Input parameters shape: (150, 23)

First 5 rows of input parameters:


Type          S1   S2          S3          S4 S5_Type         S5  \
sim                                                                     
1    GR20    0.000000  0.0    0.000000    0.000000      GS   0.000000   
2    GR20  938.857898  0.0  505.979207    0.000000      GS  35.546139   
3     GR5    0.000000  0.0    0.000000  921.110789     TRE  14.000000   
4     GR5    0.000000  0.0  505.979207  921.110789      GS  21.874547   
5    GR20    0.000000  0.0  505.979207  921.110789      BC  32.811820   

        S5_Aimp S7_Type         S7  ...    S8_Aimp S9_Type         S9  \
sim                                 ...                                 
1    273.431835      GS   0.000000  ...  318.73243     TRE   0.000000   
2    273.431835      BC  22.115649  ...  318.73243      BC  35.424723   
3    259.431835      GS  10.052568  ...  318.73243      GS  22.543005   
4    273.431835      BC   6.031541  ...  318.73243      GS  35.424723   
5    273.431835      GS  12.063081  ...  283.73243     TRE  14.000000   

        S9_Aimp S10_Type        S10    S10_Aimp S11_Type        S11  \
sim                                                                   
1    322.042932       BC   0.000000  306.671996       GS   0.000000   
2    322.042932       BC  46.000799  306.671996       GS  15.047229   
3    322.042932      TRE  21.000000  285.671996       BC  25.795249   
4    322.042932       GS  39.867359  306.671996       BC   8.598416   
5    308.042932       BC  21.467040  306.671996       BC  25.795249   

       S11_Aimp  
sim              
1    214.960412  
2    214.960412  
3    214.960412  
4    214.960412  
5    214.960412  

[5 rows x 23 columns]


LID Area columns to use: ['S1', 'S2', 'S3', 'S4', 'S5_Type', 'S5', 'S7_Type', 'S7', 'S8_Type', 'S8', 'S9_Type', 'S9', 'S10_Type', 'S10', 'S11_Type', 'S11']

Processed input features for regression:
Shape: (150, 18)


S1   S2          S3          S4 S5_Type         S5 S7_Type  \
sim                                                                       
1      0.000000  0.0    0.000000    0.000000      GS   0.000000      GS   
2    938.857898  0.0  505.979207    0.000000      GS  35.546139      BC   
3      0.000000  0.0    0.000000  921.110789     TRE  14.000000      GS   
4      0.000000  0.0  505.979207  921.110789      GS  21.874547      BC   
5      0.000000  0.0  505.979207  921.110789      BC  32.811820      GS   

            S7 S8_Type         S8 S9_Type         S9 S10_Type        S10  \
sim                                                                        
1     0.000000      GS   0.000000     TRE   0.000000       BC   0.000000   
2    22.115649      GS  15.936621      BC  35.424723       BC  46.000799   
3    10.052568      GS  35.060567      GS  22.543005      TRE  21.000000   
4     6.031541      GS  15.936621      GS  35.424723       GS  39.867359   
5    12.063081     TRE  35.000000     TRE  14.000000       BC  21.467040   

    S11_Type        S11  Type_GR20  Type_GR5  
sim                                           
1         GS   0.000000       True     False  
2         GS  15.047229       True     False  
3         BC  25.795249      False      True  
4         BC   8.598416      False      True  
5         BC  25.795249       True     False

### **Build Regression Models for the selected Objectives**